# Klasifikasi Penyakit Makula Mata Menggunakan CNN
## Deteksi CNV, DME, dan DRUSEN

Notebook ini mengimplementasikan model CNN untuk mengklasifikasikan penyakit pada makula mata dari gambar OCT (Optical Coherence Tomography).

**Penyakit yang dideteksi:**
- **CNV** (Choroidal Neovascularization)
- **DME** (Diabetic Macular Edema)
- **DRUSEN**
- **NORMAL**

## 1. Import Library

In [ ]:
# Install library jika belum terinstall
# !pip install tensorflow pillow matplotlib numpy pandas scikit-learn opencv-python seaborn --upgrade

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 2. Konfigurasi Path Dataset

Struktur folder dataset yang diharapkan:
```
dataset/
├── train/
│   ├── CNV/
│   ├── DME/
│   ├── DRUSEN/
│   └── NORMAL/
├── val/
│   ├── CNV/
│   ├── DME/
│   ├── DRUSEN/
│   └── NORMAL/
└── test/
    ├── CNV/
    ├── DME/
    ├── DRUSEN/
    └── NORMAL/
```

In [ ]:
# Konfigurasi Path Dataset
BASE_DIR = 'dataset\d3'  # Ubah sesuai lokasi dataset Anda
TRAIN_DIR = os.path.join(BASE_DIR, 'train\\train')
VAL_DIR = os.path.join(BASE_DIR, 'validation\\validation')
TEST_DIR = os.path.join(BASE_DIR, 'test\\test')

# Verifikasi path dataset
print("Verifikasi Path Dataset:")
print(f"Train Directory: {os.path.exists(TRAIN_DIR)} - {TRAIN_DIR}")
print(f"Validation Directory: {os.path.exists(VAL_DIR)} - {VAL_DIR}")
print(f"Test Directory: {os.path.exists(TEST_DIR)} - {TEST_DIR}")

# Daftar kelas penyakit
CLASSES = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
NUM_CLASSES = len(CLASSES)

print(f"\nKelas yang akan diprediksi: {CLASSES}")
print(f"Jumlah kelas: {NUM_CLASSES}")

## 3. Hyperparameter dan Konfigurasi Model

In [ ]:
# Hyperparameter
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0001

# Seed untuk reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("Konfigurasi Model:")
print(f"Ukuran Gambar: {IMG_HEIGHT}x{IMG_WIDTH}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Learning Rate: {LEARNING_RATE}")

## 4. Eksplorasi Dataset

In [ ]:
def count_images_in_directory(directory):
    """Menghitung jumlah gambar per kelas dalam direktori"""
    counts = {}
    total = 0
    
    if not os.path.exists(directory):
        print(f"Direktori {directory} tidak ditemukan!")
        return counts, total
    
    for class_name in CLASSES:
        class_path = os.path.join(directory, class_name)
        if os.path.exists(class_path):
            image_files = [f for f in os.listdir(class_path) 
                          if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
            count = len(image_files)
            counts[class_name] = count
            total += count
        else:
            counts[class_name] = 0
    
    return counts, total

# Hitung jumlah gambar di setiap set
train_counts, train_total = count_images_in_directory(TRAIN_DIR)
val_counts, val_total = count_images_in_directory(VAL_DIR)
test_counts, test_total = count_images_in_directory(TEST_DIR)

# Tampilkan statistik
print("\n" + "="*60)
print("STATISTIK DATASET")
print("="*60)

print(f"\n{'Kelas':<15} {'Train':<10} {'Validation':<15} {'Test':<10}")
print("-"*60)
for class_name in CLASSES:
    print(f"{class_name:<15} {train_counts.get(class_name, 0):<10} "
          f"{val_counts.get(class_name, 0):<15} {test_counts.get(class_name, 0):<10}")

print("-"*60)
print(f"{'TOTAL':<15} {train_total:<10} {val_total:<15} {test_total:<10}")
print("="*60)

In [ ]:
# Visualisasi distribusi dataset
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

datasets = [
    ('Training Set', train_counts),
    ('Validation Set', val_counts),
    ('Test Set', test_counts)
]

for idx, (title, counts) in enumerate(datasets):
    classes = list(counts.keys())
    values = list(counts.values())
    
    axes[idx].bar(classes, values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    axes[idx].set_title(title, fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Kelas', fontsize=12)
    axes[idx].set_ylabel('Jumlah Gambar', fontsize=12)
    axes[idx].grid(axis='y', alpha=0.3)
    
    # Tambahkan nilai di atas bar
    for i, v in enumerate(values):
        axes[idx].text(i, v + max(values)*0.02, str(v), 
                      ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 5. Fungsi Preprocessing (Konversi ke Grayscale)

In [ ]:
def preprocess_image(image):
    """
    Preprocessing gambar: konversi ke grayscale dan normalisasi
    Input: gambar RGB (H, W, 3)
    Output: gambar grayscale (H, W, 1)
    """
    # Konversi ke grayscale menggunakan weighted average (RGB to Gray)
    # Formula: Gray = 0.299*R + 0.587*G + 0.114*B
    gray = tf.image.rgb_to_grayscale(image)
    
    # Normalisasi ke range [0, 1]
    gray = gray / 255.0
    
    return gray

def preprocess_for_prediction(image_path):
    """
    Preprocessing untuk prediksi gambar baru
    """
    # Load gambar
    img = cv2.imread(image_path)
    
    if img is None:
        raise ValueError(f"Tidak dapat membaca gambar: {image_path}")
    
    # Resize
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    
    # Konversi BGR ke RGB (OpenCV menggunakan BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Konversi ke grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Expand dimensions untuk channel
    gray = np.expand_dims(gray, axis=-1)
    
    # Normalisasi
    gray = gray / 255.0
    
    # Expand dimensions untuk batch
    gray = np.expand_dims(gray, axis=0)
    
    return gray, img

print("Fungsi preprocessing telah didefinisikan.")

## 6. Data Generator dengan Preprocessing

In [ ]:
# Data Generator tanpa augmentasi (sesuai permintaan)
# Hanya melakukan rescaling karena preprocessing grayscale dilakukan di model
train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=None  # Preprocessing dilakukan di layer model
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED,
    color_mode='rgb'  # Load sebagai RGB, akan dikonversi ke grayscale di model
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    color_mode='rgb'
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    color_mode='rgb'
)

print(f"\nData berhasil dimuat!")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Test samples: {test_generator.samples}")
print(f"\nClass indices: {train_generator.class_indices}")

## 7. Visualisasi Sample Data

In [ ]:
# Visualisasi beberapa sample dari training set
def visualize_samples(generator, num_samples=8):
    """Visualisasi sample gambar dari generator"""
    images, labels = next(generator)
    
    # Konversi ke grayscale untuk visualisasi
    gray_images = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in images[:num_samples]])
    
    class_names = list(generator.class_indices.keys())
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    for idx in range(num_samples):
        axes[idx].imshow(gray_images[idx], cmap='gray')
        label_idx = np.argmax(labels[idx])
        axes[idx].set_title(f'Kelas: {class_names[label_idx]}', fontsize=12, fontweight='bold')
        axes[idx].axis('off')
    
    plt.suptitle('Sample Gambar Training Set (Grayscale)', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

visualize_samples(train_generator)

## 8. Arsitektur Model CNN

In [ ]:
def create_cnn_model():
    """
    Membuat arsitektur CNN untuk klasifikasi penyakit makula mata
    Input: RGB image (224, 224, 3)
    Output: 4 classes (CNV, DME, DRUSEN, NORMAL)
    """
    model = models.Sequential([
        # Input layer
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        
        # Preprocessing layer: Konversi RGB ke Grayscale
        layers.Lambda(lambda x: tf.image.rgb_to_grayscale(x), name='rgb_to_grayscale'),
        
        # Convolutional Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_1'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_2'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), name='pool1'),
        layers.Dropout(0.25),
        
        # Convolutional Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_1'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_2'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), name='pool2'),
        layers.Dropout(0.25),
        
        # Convolutional Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_1'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_2'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), name='pool3'),
        layers.Dropout(0.3),
        
        # Convolutional Block 4
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv4_1'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv4_2'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), name='pool4'),
        layers.Dropout(0.3),
        
        # Fully Connected Layers
        layers.Flatten(),
        layers.Dense(512, activation='relu', name='fc1'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu', name='fc2'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Output Layer
        layers.Dense(NUM_CLASSES, activation='softmax', name='output')
    ])
    
    return model

# Buat model
model = create_cnn_model()

# Compile model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Tampilkan summary model
model.summary()

In [ ]:
# Visualisasi arsitektur model
tf.keras.utils.plot_model(
    model,
    to_file='model_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    dpi=96
)

print("Arsitektur model telah disimpan sebagai 'model_architecture.png'")

## 9. Callbacks untuk Training

In [ ]:
# Buat direktori untuk menyimpan model
os.makedirs('models', exist_ok=True)

# Callbacks
callbacks = [
    # Early Stopping: hentikan training jika tidak ada improvement
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Model Checkpoint: simpan model terbaik
    ModelCheckpoint(
        filepath='models/best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Reduce Learning Rate: kurangi learning rate jika plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("Callbacks telah dikonfigurasi.")

## 10. Training Model

In [ ]:
# Mulai training
print("Memulai training model...\n")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("TRAINING SELESAI!")
print("="*60)

## 11. Visualisasi Training History

In [ ]:
def plot_training_history(history):
    """Visualisasi accuracy dan loss selama training"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend(loc='lower right')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Loss
    axes[0, 1].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, 1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend(loc='upper right')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Precision
    axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2)
    axes[1, 0].plot(history.history['val_precision'], label='Validation Precision', linewidth=2)
    axes[1, 0].set_title('Model Precision', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend(loc='lower right')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2)
    axes[1, 1].plot(history.history['val_recall'], label='Validation Recall', linewidth=2)
    axes[1, 1].set_title('Model Recall', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend(loc='lower right')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_training_history(history)

## 12. Evaluasi Model pada Test Set

In [ ]:
# Evaluasi model pada test set
print("Evaluasi model pada test set...\n")

test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator, verbose=1)

print("\n" + "="*60)
print("HASIL EVALUASI TEST SET")
print("="*60)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"F1-Score: {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}")
print("="*60)

## 13. Confusion Matrix dan Classification Report

In [ ]:
# Prediksi pada test set
test_generator.reset()
y_pred_probs = model.predict(test_generator, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASSES, yticklabels=CLASSES,
            cbar_kws={'label': 'Jumlah Prediksi'})
plt.title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Classification Report
print("\n" + "="*60)
print("CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_true, y_pred, target_names=CLASSES, digits=4))
print("="*60)

## 14. Visualisasi Prediksi pada Test Set

In [ ]:
def visualize_predictions(generator, model, num_samples=12):
    """Visualisasi prediksi model pada sample test set"""
    generator.reset()
    images, labels = next(generator)
    
    predictions = model.predict(images[:num_samples])
    
    # Konversi ke grayscale untuk visualisasi
    gray_images = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in images[:num_samples]])
    
    class_names = list(generator.class_indices.keys())
    
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.ravel()
    
    for idx in range(num_samples):
        axes[idx].imshow(gray_images[idx], cmap='gray')
        
        true_label = class_names[np.argmax(labels[idx])]
        pred_label = class_names[np.argmax(predictions[idx])]
        confidence = np.max(predictions[idx]) * 100
        
        color = 'green' if true_label == pred_label else 'red'
        
        axes[idx].set_title(
            f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.1f}%',
            fontsize=10,
            color=color,
            fontweight='bold'
        )
        axes[idx].axis('off')
    
    plt.suptitle('Prediksi Model pada Test Set', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('test_predictions.png', dpi=300, bbox_inches='tight')
    plt.show()

visualize_predictions(test_generator, model)

## 15. Simpan Model

In [ ]:
# Simpan model dalam berbagai format
print("Menyimpan model...\n")

# Format Keras (recommended)
model.save('models/retinal_disease_model.keras')
print("✓ Model disimpan sebagai: models/retinal_disease_model.keras")

# SavedModel format (untuk deployment)
model.save('models/retinal_disease_savedmodel')
print("✓ Model disimpan sebagai: models/retinal_disease_savedmodel")

# Simpan weights saja
model.save_weights('models/model_weights.weights.h5')
print("✓ Weights disimpan sebagai: models/model_weights.weights.h5")

print("\nSemua model berhasil disimpan!")

## 16. Fungsi Prediksi untuk Gambar Baru

In [ ]:
def predict_new_image(image_path, model):
    """
    Prediksi penyakit pada gambar baru
    Gambar akan otomatis dikonversi ke grayscale sebelum prediksi
    
    Args:
        image_path: Path ke gambar yang akan diprediksi
        model: Model yang sudah ditraining
    
    Returns:
        predicted_class: Kelas yang diprediksi
        confidence: Tingkat kepercayaan prediksi
        all_probabilities: Probabilitas untuk semua kelas
    """
    try:
        # Preprocessing gambar
        processed_img, original_img = preprocess_for_prediction(image_path)
        
        # Prediksi
        predictions = model.predict(processed_img, verbose=0)
        
        # Hasil prediksi
        predicted_class_idx = np.argmax(predictions[0])
        predicted_class = CLASSES[predicted_class_idx]
        confidence = predictions[0][predicted_class_idx] * 100
        
        # Probabilitas semua kelas
        all_probabilities = {CLASSES[i]: predictions[0][i] * 100 for i in range(NUM_CLASSES)}
        
        return predicted_class, confidence, all_probabilities, original_img, processed_img
    
    except Exception as e:
        print(f"Error saat memproses gambar: {str(e)}")
        return None, None, None, None, None

def visualize_prediction(image_path, model):
    """
    Visualisasi hasil prediksi untuk gambar baru
    """
    predicted_class, confidence, all_probs, original_img, processed_img = predict_new_image(image_path, model)
    
    if predicted_class is None:
        return
    
    # Plot hasil
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Gambar original
    axes[0].imshow(original_img)
    axes[0].set_title('Gambar Original (RGB)', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Gambar grayscale
    axes[1].imshow(processed_img[0, :, :, 0], cmap='gray')
    axes[1].set_title('Gambar Grayscale (Input Model)', fontsize=12, fontweight='bold')
    axes[1].axis('off')
    
    # Bar chart probabilitas
    classes = list(all_probs.keys())
    probabilities = list(all_probs.values())
    colors = ['#FF6B6B' if c == predicted_class else '#95A5A6' for c in classes]
    
    bars = axes[2].barh(classes, probabilities, color=colors)
    axes[2].set_xlabel('Probabilitas (%)', fontsize=12)
    axes[2].set_title('Probabilitas Prediksi', fontsize=12, fontweight='bold')
    axes[2].set_xlim(0, 100)
    
    # Tambahkan nilai di ujung bar
    for i, (bar, prob) in enumerate(zip(bars, probabilities)):
        axes[2].text(prob + 2, i, f'{prob:.2f}%', 
                    va='center', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print hasil
    print("\n" + "="*60)
    print("HASIL PREDIKSI")
    print("="*60)
    print(f"Gambar: {os.path.basename(image_path)}")
    print(f"Prediksi: {predicted_class}")
    print(f"Tingkat Kepercayaan: {confidence:.2f}%")
    print("\nProbabilitas untuk semua kelas:")
    for class_name, prob in sorted(all_probs.items(), key=lambda x: x[1], reverse=True):
        print(f"  {class_name}: {prob:.2f}%")
    print("="*60)

print("Fungsi prediksi telah didefinisikan.")

## 17. Contoh Prediksi Gambar Baru

In [ ]:
# Contoh penggunaan prediksi gambar baru
# Ganti path dengan gambar yang ingin diprediksi

# Contoh 1: Prediksi satu gambar
NEW_IMAGE_PATH = 'path/to/your/new_image.jpg'  # Ubah dengan path gambar Anda

# Uncomment baris di bawah untuk menjalankan prediksi
# visualize_prediction(NEW_IMAGE_PATH, model)

print("Untuk memprediksi gambar baru:")
print("1. Ubah variable NEW_IMAGE_PATH dengan path gambar Anda")
print("2. Uncomment baris visualize_prediction()")
print("3. Jalankan cell ini")

## 18. Batch Prediction untuk Multiple Images

In [ ]:
def batch_predict(image_folder, model, output_csv='predictions.csv'):
    """
    Prediksi batch untuk multiple images dalam satu folder
    
    Args:
        image_folder: Folder berisi gambar-gambar yang akan diprediksi
        model: Model yang sudah ditraining
        output_csv: Nama file CSV untuk menyimpan hasil
    """
    results = []
    
    # Ambil semua file gambar
    image_files = [f for f in os.listdir(image_folder) 
                   if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    
    print(f"Ditemukan {len(image_files)} gambar untuk diprediksi...\n")
    
    for idx, image_file in enumerate(image_files, 1):
        image_path = os.path.join(image_folder, image_file)
        
        predicted_class, confidence, all_probs, _, _ = predict_new_image(image_path, model)
        
        if predicted_class:
            result = {
                'filename': image_file,
                'predicted_class': predicted_class,
                'confidence': f"{confidence:.2f}%"
            }
            
            # Tambahkan probabilitas untuk setiap kelas
            for class_name, prob in all_probs.items():
                result[f'prob_{class_name}'] = f"{prob:.2f}%"
            
            results.append(result)
            print(f"[{idx}/{len(image_files)}] {image_file}: {predicted_class} ({confidence:.2f}%)")
    
    # Simpan ke CSV
    if results:
        df = pd.DataFrame(results)
        df.to_csv(output_csv, index=False)
        print(f"\n✓ Hasil prediksi disimpan ke: {output_csv}")
        return df
    else:
        print("Tidak ada prediksi yang berhasil.")
        return None

# Contoh penggunaan
# NEW_IMAGES_FOLDER = 'path/to/your/images/folder'
# predictions_df = batch_predict(NEW_IMAGES_FOLDER, model)
# print(predictions_df)

print("Fungsi batch prediction telah didefinisikan.")

## 19. Load Model yang Sudah Disimpan

In [ ]:
# Untuk menggunakan model yang sudah disimpan di lain waktu
def load_trained_model(model_path='models/best_model.keras'):
    """
    Load model yang sudah ditraining sebelumnya
    """
    try:
        loaded_model = tf.keras.models.load_model(model_path)
        print(f"✓ Model berhasil dimuat dari: {model_path}")
        return loaded_model
    except Exception as e:
        print(f"Error saat memuat model: {str(e)}")
        return None

# Contoh penggunaan:
# loaded_model = load_trained_model('models/best_model.keras')
# visualize_prediction('path/to/image.jpg', loaded_model)

print("Fungsi load model telah didefinisikan.")

## 20. Summary dan Informasi Penggunaan

In [ ]:
print("\n" + "="*80)
print("SUMMARY - SISTEM KLASIFIKASI PENYAKIT MAKULA MATA")
print("="*80)
print("\n📋 INFORMASI MODEL:")
print(f"  • Jenis Model: Convolutional Neural Network (CNN)")
print(f"  • Input: Gambar RGB (224x224x3) → Otomatis dikonversi ke Grayscale")
print(f"  • Output: 4 Kelas (CNV, DME, DRUSEN, NORMAL)")
print(f"  • Framework: TensorFlow {tf.__version__} / Keras {keras.__version__}")

print("\n📊 PERFORMA MODEL:")
print(f"  • Test Accuracy: {test_accuracy*100:.2f}%")
print(f"  • Test Precision: {test_precision:.4f}")
print(f"  • Test Recall: {test_recall:.4f}")

print("\n💾 FILE YANG DISIMPAN:")
print("  • models/best_model.keras - Model terbaik")
print("  • models/retinal_disease_model.keras - Model final")
print("  • models/model_weights.weights.h5 - Weights model")
print("  • training_history.png - Grafik training")
print("  • confusion_matrix.png - Confusion matrix")
print("  • test_predictions.png - Visualisasi prediksi")

print("\n🔧 CARA MENGGUNAKAN UNTUK PREDIKSI:")
print("""  
  # 1. Load model
  model = load_trained_model('models/best_model.keras')
  
  # 2. Prediksi satu gambar
  visualize_prediction('path/to/image.jpg', model)
  
  # 3. Prediksi batch (multiple images)
  batch_predict('path/to/folder/', model, 'results.csv')
""")

print("\n⚠️  CATATAN PENTING:")
print("  • Gambar baru akan OTOMATIS dikonversi ke GRAYSCALE sebelum prediksi")
print("  • Pastikan gambar memiliki resolusi yang cukup baik")
print("  • Model telah dioptimalkan untuk gambar OCT retina")
print("  • Tidak ada augmentasi data (sesuai permintaan)")

print("\n" + "="*80)
print("✓ NOTEBOOK SELESAI!")
print("="*80 + "\n")